In [1]:
import pandas as pd
import numpy as np

In [2]:
df_assessements = pd.read_csv('assessments.csv')
df_courses = pd.read_csv('courses.csv')
df_student_assessement = pd.read_csv('studentAssessment.csv')
df_student_registration = pd.read_csv('studentRegistration.csv')

In [3]:
df_courses.head()

,code_module,code_presentation,module_presentation_length
0,AAA,2013J,268
1,AAA,2014J,269
2,BBB,2013J,268
3,BBB,2014J,262
4,BBB,2013B,240


In [5]:
df_assessements.head()

,code_module,code_presentation,id_assessment,assessment_type,date,weight
0,AAA,2013J,1752,TMA,19.0,10.0
1,AAA,2013J,1753,TMA,54.0,20.0
2,AAA,2013J,1754,TMA,117.0,20.0
3,AAA,2013J,1755,TMA,166.0,20.0
4,AAA,2013J,1756,TMA,215.0,30.0


In [8]:
df_student_assessement.head()

,id_assessment,id_student,date_submitted,is_banked,score
0,1752,11391,18,0,78.0
1,1752,28400,22,0,70.0
2,1752,31604,17,0,72.0
3,1752,32885,26,0,69.0
4,1752,38053,19,0,79.0


In [10]:
df_student_registration.head()

,code_module,code_presentation,id_student,date_registration,date_unregistration
0,AAA,2013J,11391,-159.0,NaN
1,AAA,2013J,28400,-53.0,NaN
2,AAA,2013J,30268,-92.0,12.0
3,AAA,2013J,31604,-52.0,NaN
4,AAA,2013J,32885,-176.0,NaN


### 1. Сколько студентов успешно сдали только один курс? (Успешная сдача — это зачёт по курсу на экзамене) 

In [12]:
df_exams = df_assessements[df_assessements.assessment_type == 'Exam']

In [13]:
df_score_subject = df_student_assessement \
    .merge(df_exams, on='id_assessment') \

In [14]:
df_students_success_exams_is_one = df_score_subject \
    [(df_score_subject.assessment_type == 'Exam') & (df_score_subject.score >= 40)] \
    .groupby('id_student', as_index=False) \
    .agg({'id_assessment': 'count'}) \
    .rename(columns={'id_assessment': 'success_exams'}) \
    .query('success_exams == 1') \
    .id_student.nunique()

In [15]:
df_students_success_exams_is_one

3802

### 2. Выяви самый сложный и самый простой экзамен: найди курсы и экзамены в рамках курса, которые обладают самой низкой и самой высокой завершаемостью*.
### *завершаемость = кол-во успешных экзаменов / кол-во всех попыток сдать экзамен

In [16]:
df_exams_all_results = df_score_subject \
    .groupby(['code_module', 'id_assessment'], as_index=False) \
    .agg({'id_student': 'count'})

In [17]:
df_exams_good_results = df_score_subject \
    .query('score >= 40') \
    .groupby(['code_module', 'id_assessment'], as_index=False) \
    .agg({'id_student': 'count'})

In [18]:
df_good_to_all = df_exams_good_results.merge(df_exams_all_results, on=['code_module','id_assessment']) 

In [19]:
df_good_to_all['completeness'] = df_good_to_all['id_student_x'] / df_good_to_all['id_student_y']

In [20]:
df_good_to_all.sort_values('completeness', ascending=False)

,code_module,id_assessment,id_student_x,id_student_y,completeness
4,DDD,25361,485,524,0.925573
3,DDD,25354,878,968,0.907025
0,CCC,24290,664,747,0.888889
5,DDD,25368,842,950,0.886316
1,CCC,24299,1019,1168,0.872432
2,DDD,25340,504,602,0.837209


### 3. По каждому предмету определи средний срок сдачи экзаменов (под сдачей понимаем последнее успешное прохождение экзамена студентом). 

In [21]:
df_score_subject.query('score >= 40') \
    .groupby('code_module', as_index=False) \
    .agg({'date_submitted': 'mean'})

,code_module,date_submitted
0,CCC,239.345217
1,DDD,237.976006


### 4. Выяви самые популярные курсы (ТОП-3) по количеству регистраций на них. А также курсы с самым большим оттоком (ТОП-3). 

In [22]:
top_3_courses_registration = df_student_registration.groupby('code_module', as_index=False) \
    .agg({'date_registration': 'count'}) \
    .rename(columns={'date_registration': 'students_registered'}) \
    .sort_values('students_registered', ascending=False) \
    .head(3)

In [23]:
top_3_courses_unregistration = df_student_registration.groupby('code_module', as_index=False) \
    .agg({'date_unregistration': 'count'}) \
    .rename(columns={'date_unregistration': 'students_unregistered'}) \
    .sort_values('students_unregistered', ascending=False) \
    .head(3)

### 5.Напиши функцию на python, позволяющую строить когортный (семестровый) анализ. В период с начала 2013 по конец 2014 выяви семестр с самой низкой завершаемостью курсов и самыми долгими средними сроками сдачи курсов. 

In [28]:
# сначала попробуем решитть классически

In [29]:
df_semester_unfinished = df_student_registration.groupby('code_presentation', as_index=False) \
    .agg({'date_unregistration': 'count'}) \
    .rename(columns={'date_unregistration': 'students_unregistered'}) \
    .sort_values('students_unregistered', ascending=False)

In [30]:
df_big = df_student_assessement.merge(df_assessements, on='id_assessment')

In [31]:
df_avg_submitted_exam_passed = df_big.query('assessment_type == "Exam" and score >= 40 ') \
    .groupby(['code_presentation','code_module'], as_index=False) \
    .agg({'date_submitted': 'mean'}) \
    .rename(columns={'date_submitted':'avg_submitted'}) \
    .sort_values('avg_submitted', ascending=False) \
    .head()

In [32]:
df_worst_semester = df_semester_unfinished.merge(df_avg_submitted_exam_passed, on='code_presentation')

In [33]:
# то же самое, только засунутое в функцию

In [34]:
def worst_semesters(df_big):
    
    df_avg_submitted_exam_passed = df_big.query('assessment_type == "Exam" and score >= 40 ') \
    .groupby(['code_presentation','code_module'], as_index=False) \
    .agg({'date_submitted': 'mean'}) \
    .rename(columns={'date_submitted':'avg_submitted'}) \
    .sort_values('avg_submitted', ascending=False) \
    .head()
    
    df_semester_unfinished = df_student_registration.groupby('code_presentation', as_index=False) \
    .agg({'date_unregistration': 'count'}) \
    .rename(columns={'date_unregistration': 'students_unregistered'}) \
    .sort_values('students_unregistered', ascending=False)
    
    df_worst_semester = df_semester_unfinished.merge(df_avg_submitted_exam_passed, on='code_presentation')
    
    return df_worst_semester

In [35]:
worst_semesters(df_big)

,code_presentation,students_unregistered,code_module,avg_submitted
0,2014J,3739,CCC,244.404318
1,2014J,3739,DDD,242.804038
2,2014B,2610,DDD,234.936082
3,2014B,2610,CCC,231.581325
4,2013J,2376,DDD,239.509112


### 6. Часто для качественного анализа аудитории использую подходы, основанные на сегментации. Используя python, построй адаптированные RFM-кластеры студентов, чтобы качественно оценить свою аудиторию. В адаптированной кластеризации можешь выбрать следующие метрики: R - среднее время сдачи одного экзамена, F - завершаемость курсов, M - среднее количество баллов, получаемое за экзамен. Подробно опиши, как ты создавал кластеры. Для каждого RFM-сегмента построй границы метрик recency, frequency и monetary для интерпретации этих кластеров. Пример такого описания: RFM-сегмент 132 (recency=1, frequency=3, monetary=2) имеет границы метрик recency от 130 до 500 дней, frequency от 0.1 до 0.5, monetary от 55 до 72 баллов.

In [36]:
df_exam = df_big.query('assessment_type == "Exam"')

In [37]:
df_avg_date = df_exam.groupby(['id_student', 'code_module'], as_index=False) \
    .agg({'date_submitted':'mean'}) \
    .rename(columns={'date_submitted': 'avg_date'})

In [38]:
df_avg_score = df_exam.groupby(['id_student', 'code_module'], as_index=False) \
    .agg({'score':'mean'}) \
    .rename(columns={'score': 'avg_score'})

In [39]:
df_exams_all_results = df_exam \
    .groupby(['code_module', 'id_assessment'], as_index=False) \
    .agg({'id_student': 'count'}) \
    .rename(columns={'id_student': 'all_students'})

In [40]:
df_exams_good_results = df_exam \
    .query('score >= 40') \
    .groupby(['code_module', 'id_assessment'], as_index=False) \
    .agg({'id_student': 'count'}) \
    .rename(columns={'id_student': 'students_survived'})

In [41]:
df_exam_good_to_all = df_exams_good_results.merge(df_exams_all_results, on=['code_module', 'id_assessment'])
df_exam_good_to_all['completeness'] = df_exam_good_to_all['students_survived'] / df_exam_good_to_all['all_students']
df_exam_good_to_all = df_exam_good_to_all[['code_module', 'id_assessment','completeness']]

In [42]:
df_for_rfm = df_avg_date.merge(df_avg_score, on=['id_student', 'code_module']) \
            .merge(df_exam_good_to_all, on='code_module')

In [43]:
df_for_rfm = df_for_rfm[['id_student', 'code_module', 'id_assessment', 'avg_date', 'completeness', 'avg_score']]

In [44]:
df_for_rfm.describe()

,id_student,id_assessment,avg_date,completeness,avg_score
count,1.599000e+04,15990.000000,15990.000000,15990.000000,15990.000000
mean,7.248491e+05,25101.554409,238.264165,0.887026,64.796123
std,5.781751e+05,453.044007,5.460209,0.029265,19.943905
min,2.369800e+04,24290.000000,229.000000,0.837209,0.000000
25%,4.991035e+05,25340.000000,234.000000,0.872432,50.000000
50%,5.854320e+05,25354.000000,241.000000,0.887602,64.000000
75%,6.440092e+05,25361.000000,243.000000,0.907025,80.000000
max,2.698251e+06,25368.000000,285.000000,0.925573,100.000000


In [45]:
df_for_rfm['R'] = ''
df_for_rfm['F'] = ''
df_for_rfm['M'] = ''

In [46]:
# Все значения будем делить по квантилям 
# Всего три возможных варианта: < q=0.25, > q=0.75, и то что между (0.25-0.75)

In [47]:
# Чем быстрее сдают (чем меньше значение), тем лучше. Значит, 1 = (avg_date < q=0.25)

In [48]:
avg_date_quantile_25 = df_for_rfm.avg_date.quantile(q=0.25)
avg_date_quantile_75 = df_for_rfm.avg_date.quantile(q=0.75)
df_for_rfm.loc[df_for_rfm['avg_date'] < avg_date_quantile_25, 'R'] = 1
df_for_rfm.loc[df_for_rfm['avg_date'] > avg_date_quantile_75, 'R'] = 3
df_for_rfm.R = df_for_rfm.R.replace('', 2)

In [49]:
# Чем больше студентов завершают курс (чем выше процент сдачи), тем лучше. Значит, 1 = (completeness >  q=0.75)

In [50]:
completeness_quantile_25 = df_for_rfm.completeness.quantile(q=0.25)
completeness_quantile_75 = df_for_rfm.completeness.quantile(q=0.75)
df_for_rfm.loc[df_for_rfm['completeness'] > completeness_quantile_75, 'F'] = 1
df_for_rfm.loc[df_for_rfm['completeness'] < completeness_quantile_25, 'F'] = 3
df_for_rfm.F = df_for_rfm.F.replace('', 2)

In [51]:
# Чем выше средняя оценка (чем больше число), тем лучше. Значит 1 = (avg_score > q=0.75)

In [52]:
avg_score_quantile_25 = df_for_rfm.avg_score.quantile(q=0.25)
avg_score_quantile_75 = df_for_rfm.avg_score.quantile(q=0.75)
df_for_rfm.loc[df_for_rfm['avg_score'] > avg_score_quantile_75, 'M'] = 1
df_for_rfm.loc[df_for_rfm['avg_score'] < avg_score_quantile_25 , 'M'] = 3
df_for_rfm.M = df_for_rfm.M.replace('', 2)

In [53]:
df_for_rfm.head()

,id_student,code_module,id_assessment,avg_date,completeness,avg_score,R,F,M
0,23698,CCC,24290,243.0,0.888889,80.0,2,2,2
1,23698,CCC,24299,243.0,0.872432,80.0,2,2,2
2,27116,CCC,24290,243.0,0.888889,96.0,2,2,1
3,27116,CCC,24299,243.0,0.872432,96.0,2,2,1
4,28787,CCC,24290,243.0,0.888889,44.0,2,2,3
